Se dorește scrierea unei aplicații de image retrieval care găsește cea mai similară imagine cu o imagine dată (query) dintr-o bază de date. În acest sens, trebuie construit un autoencoder care va învăța particularitățile bazei de date de antrenare. După aceea, pornind de la imaginea query, se va căuta în baza de date de test imaginea cu descriptorul de trăsături cel mai asemănător. Descriptorul va fi ales ca fiind vectorul latent/bottleneck al autoencoder-ului, iar metoda de comparație între descriptori este distanța Euclidiană. Succint, pașii ce trebuie urmați sunt:
- Antrenarea unui autoencoder
- Alegerea unei imagini aleatoare din baza de date de test
- Extragerea descriptorului de trăsături (vectorul latent obținut cu autoencoder) al imaginii query;
- Comparația descriptorului query cu toți ceilalți descriptori existenți în baza de date de test;
- Afișarea perechii de imagini cu caracteristicile cele mai asemănătoare.


### TODO: ###
1. (10p) Completarea arhitecturii autoencoderului cu cate un strat suplimentar pe ultima poziție a codorului, respectiv prima poziție a decodorului. Acest strat va fi de convoluție/deconvoluție și va urmări același tipar cu restul straturilor: pentru codor, va înjumătăți numărul de canale, iar pentru decodor va dubla numărul de canale.
2. (10p) Completarea clasei Autoencoder cu codul necesar interacțiunii dintre codor și decodor;
3. (10p) Extragerea descriptorilor de trăsături și stocarea lor pentru toate imaginile din baza de date de test;
4. (10p) Scrierea funcției de calcul a distanței Euclidiene;
5. (10p) Afișarea în paralel a imaginii query cu top-3 cele mai asemănătoare imagini din baza de date de test.

In [ ]:
import sys

%matplotlib inline
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt

latent_dims = 10
num_epochs = 10
batch_size = 128
capacity = 64
learning_rate = 1e-3
variational_beta = 1

In [ ]:
# Descarcarea bazei de date MNIST Digits
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

img_transform = transforms.Compose([transforms.ToTensor()])

train_dataset = MNIST(root='./data/MNIST', download=True, train=True, transform=img_transform)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = MNIST(root='./data/MNIST', download=True, train=False, transform=img_transform)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# hiperparametri
latent_dims = 10
num_epochs = 3
batch_size = 128
capacity = 64
learning_rate = 1e-2
use_gpu = True

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        c = capacity
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=c, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels=c, out_channels=c*2, kernel_size=4, stride=2, padding=1)
        # de adaugat self.conv3
        self.fc = nn.Linear(in_features=c*4*4*4, out_features=latent_dims)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1) # flatten batch of multi-channel feature maps to a batch of feature vectors
        x = self.fc(x)
        return x

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        c = capacity
        self.fc = nn.Linear(in_features=latent_dims, out_features=c*4*4*4)
        # de adaugat self.conv3
        self.conv2 = nn.ConvTranspose2d(in_channels=c*2, out_channels=c, kernel_size=4, stride=2, padding=1)
        self.conv1 = nn.ConvTranspose2d(in_channels=c, out_channels=1, kernel_size=4, stride=2, padding=1)

    def forward(self, x):
        x = self.fc(x)
        x = x.view(x.size(0), capacity*4, 4, 4) # unflatten batch of feature vectors to a batch of multi-channel feature maps
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv2(x))
        x = torch.tanh(self.conv1(x)) # last layer before output is tanh, since the images are normalized and 0-centered
        return x

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
    # de adaugat restul functionalitatii autoencoder-ului

autoencoder = Autoencoder()

device = torch.device("cuda:0" if use_gpu and torch.cuda.is_available() else "cpu")
autoencoder = autoencoder.to(device)

In [ ]:
# functie de afisare a unei singure imagini
def display(img, img_size=28, title=""):
    plt.imshow(img.cpu().detach().numpy().reshape((img_size, img_size)), cmap='gray')
    plt.title(title)
    plt.show()

In [ ]:
optimizer = torch.optim.Adam(params=autoencoder.parameters(), lr=learning_rate, weight_decay=1e-5)
loss = nn.MSELoss()

# setare in modul de antrenare
autoencoder.train()

train_loss_avg = []

print('Training ...')
for epoch in range(num_epochs):
    train_loss_avg.append(0)
    num_batches = 0

    for image_batch, _ in train_dataloader:
        image_batch = image_batch.to(device)

        # autoencoder reconstruction
        image_batch_recon = autoencoder(image_batch)

        # reconstruction error
        reconstruction_loss = loss(image_batch_recon, image_batch)

        # backpropagation
        optimizer.zero_grad()
        reconstruction_loss.backward()

        # one step of the optmizer (using the gradients from backpropagation)
        optimizer.step()

        train_loss_avg[-1] += reconstruction_loss.item()
        num_batches += 1

    train_loss_avg[-1] /= num_batches
    print('Epoca [%d / %d] Eroare de reconstructie medie: %f' % (epoch+1, num_epochs, train_loss_avg[-1]))